# Interactive Dashboard: E-Commerce Inventory & Sales Intelligence

## Executive Summary
This project implements a comprehensive **E-Commerce Database Application** using SQLite3 and Python. It is designed to demonstrate advanced database management skills, including complex schema design, batch data processing, and interactive analytical visualization.

### Scenario Overview
We simulate a mid-sized online retail store operating across multiple regions. The system tracks:
- **Customers**: Demographics and regional distribution.
- **Products**: Inventory levels, categories, and pricing.
- **Orders**: Transactional history with detailed line items.
- **Reviews**: Customer feedback and product ratings.

### Database Schema (ERD Description)
The database consists of 5 related tables:
1. **Customers** (`CustomerID` PK): Stores user profiles.
2. **Products** (`ProductID` PK): Catalog of items with stock and price.
3. **Orders** (`OrderID` PK): Links to `Customers`. Captures order timing.
4. **OrderItems** (`OrderItemID` PK): Junction table linking `Orders` and `Products`. Captures quantity and unit price at time of purchase.
5. **Reviews** (`ReviewID` PK): Links `Customers` and `Products`. Stores text and numeric ratings.

---

## 1. Setup and Dependencies
Installing `Faker` for realistic data generation and importing necessary libraries.

In [ ]:
!pip install faker -q

import sqlite3
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configuration
DB_NAME = 'ecommerce_analytics.db'
fake = Faker()
Faker.seed(42)
random.seed(42)

## 2. Database Schema Design (DDL)
Defining the relational structure with Foreign Keys to ensure integrity.

In [ ]:
def create_schema(db_name):
    """
    Creates the SQLite database and defines the 5-table schema.
    Drops tables if they exist to ensure a fresh start.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Drop tables to reset
    cursor.executescript('''
    DROP TABLE IF EXISTS Reviews;
    DROP TABLE IF EXISTS OrderItems;
    DROP TABLE IF EXISTS Orders;
    DROP TABLE IF EXISTS Products;
    DROP TABLE IF EXISTS Customers;
    ''')
    
    # Create Tables
    schema_script = '''
    CREATE TABLE Customers (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        FullName TEXT NOT NULL,
        Email TEXT UNIQUE,
        Region TEXT CHECK(Region IN ('North', 'South', 'East', 'West', 'Central')),
        JoinDate DATE
    );

    CREATE TABLE Products (
        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
        ProductName TEXT NOT NULL,
        Category TEXT,
        Price REAL,
        StockLevel INTEGER
    );

    CREATE TABLE Orders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INTEGER,
        OrderDate DATE,
        TotalAmount REAL DEFAULT 0,
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    );

    CREATE TABLE OrderItems (
        OrderItemID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INTEGER,
        ProductID INTEGER,
        Quantity INTEGER,
        UnitPrice REAL,
        FOREIGN KEY(OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID)
    );

    CREATE TABLE Reviews (
        ReviewID INTEGER PRIMARY KEY AUTOINCREMENT,
        ProductID INTEGER,
        CustomerID INTEGER,
        Rating INTEGER CHECK(Rating BETWEEN 1 AND 5),
        Comment TEXT,
        ReviewDate DATE,
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID),
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    );
    '''
    cursor.executescript(schema_script)
    conn.commit()
    conn.close()
    print("Database schema created successfully.")

# Initialize Database
create_schema(DB_NAME)

## 3. Batch Data Simulation
Generating large volumes of realistic mock data using `Faker` and inserting it efficiently using `executemany`.

In [ ]:
def generate_data(db_name, num_customers=1000, num_products=100, num_orders=5000):
    """
    Generates and inserts batch data for the e-commerce simulation.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    print("Starting data generation...")
    
    # --- 1. Customers ---
    regions = ['North', 'South', 'East', 'West', 'Central']
    customers = []
    for _ in range(num_customers):
        customers.append((
            fake.name(),
            fake.unique.email(),
            random.choice(regions),
            fake.date_between(start_date='-2y', end_date='today')
        ))
    cursor.executemany('INSERT INTO Customers (FullName, Email, Region, JoinDate) VALUES (?, ?, ?, ?)', customers)
    print(f"Inserted {num_customers} customers.")

    # --- 2. Products ---
    categories = ['Electronics', 'Clothing', 'Home', 'Books', 'Sports']
    products = []
    for _ in range(num_products):
        products.append((
            fake.catch_phrase(),
            random.choice(categories),
            round(random.uniform(10, 500), 2),
            random.randint(0, 500)
        ))
    cursor.executemany('INSERT INTO Products (ProductName, Category, Price, StockLevel) VALUES (?, ?, ?, ?)', products)
    print(f"Inserted {num_products} products.")

    # --- 3. Orders & OrderItems ---
    # We need valid CustomerIDs and ProductIDs
    cust_ids = [row[0] for row in cursor.execute('SELECT CustomerID FROM Customers').fetchall()]
    prod_rows = cursor.execute('SELECT ProductID, Price FROM Products').fetchall()
    prod_dict = {row[0]: row[1] for row in prod_rows}
    prod_ids = list(prod_dict.keys())

    orders = []
    order_items = []
    
    for _ in range(num_orders):
        order_date = fake.date_between(start_date='-1y', end_date='today')
        cust_id = random.choice(cust_ids)
        
        # Create Order first to get ID (simulated, we insert later so we can't get ID immediately in batch)
        # Strategy: Insert Orders one by one? No, slow. 
        # Strategy: Generate data structure, insert Orders, get IDs? 
        # Optimization: We will insert Orders first, then fetch their IDs.
        orders.append((cust_id, order_date, 0)) # TotalAmount placeholder

    cursor.executemany('INSERT INTO Orders (CustomerID, OrderDate, TotalAmount) VALUES (?, ?, ?)', orders)
    
    # Fetch generated OrderIDs
    order_ids = [row[0] for row in cursor.execute('SELECT OrderID FROM Orders').fetchall()]
    
    # Generate Items for these orders
    order_totals = {}
    
    for oid in order_ids:
        num_items = random.randint(1, 5)
        current_order_total = 0
        
        for _ in range(num_items):
            pid = random.choice(prod_ids)
            qty = random.randint(1, 4)
            price = prod_dict[pid]
            line_total = price * qty
            current_order_total += line_total
            
            order_items.append((oid, pid, qty, price))
        
        order_totals[oid] = round(current_order_total, 2)

    cursor.executemany('INSERT INTO OrderItems (OrderID, ProductID, Quantity, UnitPrice) VALUES (?, ?, ?, ?)', order_items)
    print(f"Inserted {len(order_items)} order items.")

    # Update Order Totals
    # Batch update is tricky in SQLite without a temp table or many queries.
    # For simulation speed, we can use a transaction with many updates or a clever REPLACE.
    # Let's use a loop with transaction, it's acceptable for 5000 rows here.
    print("Updating order totals...")
    update_data = [(amt, oid) for oid, amt in order_totals.items()]
    cursor.executemany('UPDATE Orders SET TotalAmount = ? WHERE OrderID = ?', update_data)

    # --- 4. Reviews ---
    reviews = []
    for _ in range(2000):
        reviews.append((
            random.choice(prod_ids),
            random.choice(cust_ids),
            random.randint(1, 5),
            fake.sentence(),
            fake.date_between(start_date='-6m', end_date='today')
        ))
    cursor.executemany('INSERT INTO Reviews (ProductID, CustomerID, Rating, Comment, ReviewDate) VALUES (?, ?, ?, ?, ?)', reviews)
    print(f"Inserted {len(reviews)} reviews.")

    conn.commit()
    conn.close()
    print("Data generation complete.")

# Run Data Generation
generate_data(DB_NAME)

## 4. Advanced SQL Analysis
Implementing complex SQL queries using Python wrappers. We demonstrate:
1.  **Aggregations**: Revenue by Product Category.
2.  **Joins**: Detailed Customer Order History.
3.  **Window Functions**: Daily Revenue Running Totals and Regional Rankings.

In [ ]:
def run_query(query, db_name=DB_NAME):
    """Helper to run SQL and return DataFrame"""
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# --- Query 1: Revenue by Category (Aggregation & Join) ---
def get_revenue_by_category():
    sql = '''
    SELECT 
        p.Category,
        COUNT(oi.OrderItemID) as ItemsSold,
        SUM(oi.Quantity * oi.UnitPrice) as TotalRevenue,
        AVG(oi.UnitPrice) as AvgPrice
    FROM OrderItems oi
    JOIN Products p ON oi.ProductID = p.ProductID
    GROUP BY p.Category
    ORDER BY TotalRevenue DESC
    '''
    return run_query(sql)

# --- Query 2: Top Customers by Region (Window Function - Rank) ---
def get_top_customers_by_region():
    sql = '''
    WITH CustomerSpending AS (
        SELECT 
            c.CustomerID,
            c.FullName,
            c.Region,
            SUM(o.TotalAmount) as TotalSpent
        FROM Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        GROUP BY c.CustomerID
    ),
    RankedCustomers AS (
        SELECT 
            Region,
            FullName,
            TotalSpent,
            RANK() OVER (PARTITION BY Region ORDER BY TotalSpent DESC) as RankInRegion
        FROM CustomerSpending
    )
    SELECT *
    FROM RankedCustomers
    WHERE RankInRegion <= 3
    '''
    return run_query(sql)

# --- Query 3: Daily Revenue Trends (Window Function - Running Total) ---
def get_daily_revenue_trend():
    sql = '''
    WITH DailySales AS (
        SELECT 
            OrderDate,
            SUM(TotalAmount) as DailyTotal
        FROM Orders
        GROUP BY OrderDate
    )
    SELECT 
        OrderDate,
        DailyTotal,
        SUM(DailyTotal) OVER (ORDER BY OrderDate ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as MovingAvg7Day,
        SUM(DailyTotal) OVER (ORDER BY OrderDate) as CumulativeRevenue
    FROM DailySales
    ORDER BY OrderDate
    '''
    return run_query(sql)

# Display samples
print("--- Revenue by Category ---")
display(get_revenue_by_category())
print("\n--- Top 3 Customers per Region ---")
display(get_top_customers_by_region())

## 5. Interactive Dashboard
Using `ipywidgets` and `matplotlib` to create an interactive exploration tool. Users can filter data by Region and Category to see dynamic visualizations.

In [ ]:
def plot_dashboard(region, category):
    """
    Updates charts based on selected Region and Category.
    """
    conn = sqlite3.connect(DB_NAME)
    
    # --- Data Fetching ---
    # 1. Filtered Sales Trend
    trend_sql = f'''
    SELECT o.OrderDate, SUM(oi.Quantity * oi.UnitPrice) as Revenue
    FROM Orders o
    JOIN OrderItems oi ON o.OrderID = oi.OrderID
    JOIN Products p ON oi.ProductID = p.ProductID
    JOIN Customers c ON o.CustomerID = c.CustomerID
    WHERE c.Region = ? AND p.Category = ?
    GROUP BY o.OrderDate
    ORDER BY o.OrderDate
    '''
    df_trend = pd.read_sql_query(trend_sql, conn, params=(region, category))
    if not df_trend.empty:
        df_trend['OrderDate'] = pd.to_datetime(df_trend['OrderDate'])

    # 2. Product Performance in Region
    prod_sql = f'''
    SELECT p.ProductName, SUM(oi.Quantity) as UnitsSold
    FROM OrderItems oi
    JOIN Products p ON oi.ProductID = p.ProductID
    JOIN Orders o ON oi.OrderID = o.OrderID
    JOIN Customers c ON o.CustomerID = c.CustomerID
    WHERE c.Region = ? AND p.Category = ?
    GROUP BY p.ProductName
    ORDER BY UnitsSold DESC
    LIMIT 10
    '''
    df_prod = pd.read_sql_query(prod_sql, conn, params=(region, category))

    conn.close()

    # --- Visualization ---
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Plot 1: Sales Trend
    if not df_trend.empty:
        sns.lineplot(data=df_trend, x='OrderDate', y='Revenue', ax=axes[0], color='teal')
        axes[0].set_title(f'Daily Revenue Trend: {region} - {category}')
        axes[0].set_ylabel('Revenue ($)')
        axes[0].tick_params(axis='x', rotation=45)
    else:
        axes[0].text(0.5, 0.5, 'No Data Available', ha='center')

    # Plot 2: Top Products
    if not df_prod.empty:
        sns.barplot(data=df_prod, x='UnitsSold', y='ProductName', ax=axes[1], palette='viridis')
        axes[1].set_title(f'Top 10 Products: {region} - {category}')
    else:
        axes[1].text(0.5, 0.5, 'No Data Available', ha='center')

    plt.tight_layout()
    plt.show()

# --- Widgets Setup ---
regions = ['North', 'South', 'East', 'West', 'Central']
categories = ['Electronics', 'Clothing', 'Home', 'Books', 'Sports']

region_dropdown = widgets.Dropdown(options=regions, value='North', description='Region:')
category_dropdown = widgets.Dropdown(options=categories, value='Electronics', description='Category:')

ui = widgets.HBox([region_dropdown, category_dropdown])
out = widgets.interactive_output(plot_dashboard, {'region': region_dropdown, 'category': category_dropdown})

display(ui, out)

## 6. Reflection
**Challenges Faced**:
One of the main challenges was ensuring the data generation was realistic while maintaining referential integrity between the 5 tables. Specifically, generating `OrderItems` required careful handling to ensure they linked to valid `Orders` and `Products`. Additionally, implementing the 'Running Total' window function in SQLite required verifying the correct SQL syntax support in the Colab environment. The interactive dashboard required balancing the query complexity with performance to ensure the widgets updated smoothly.